Run with the following conda environment: `../../conda_envs/training_env`

In [ ]:
import matplotlib.pyplot as plt
import re
import numpy as np
from matplotlib.gridspec import GridSpec 
import matplotlib.pylab as pylab

In [ ]:
params = {
    'legend.fontsize': 14,
    'legend.title_fontsize': 16,
    'axes.labelsize': 18,
    'axes.titlesize': 22,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'figure.titlesize': 25,
}
plt.rc('font', size=16)
plt.rc('pdf', fonttype=42)
pylab.rcParams.update(params)

In [ ]:
coda_name = 'DUMBO'
results_dir = '../../results/simu_output'

In [ ]:
models = [
    "oracle_531KB",
    "baseline",
    "es",
    "coda",
    "pheavy",
]

memory_budgets_folders = [
    "memory_1.0MB/",
    "memory_1.5MB/",
    "memory_2.0MB/",
]

memory_budgets_folders_uni = [
    "memory_0.01MB/",
    "memory_0.015MB/",
    "memory_0.02MB/",
]

In [ ]:
def plot_errors(ax, models, memory_budgets_folders, scores, legend=True, ncol=1):    
    blue, orange, green, red, violet = plt.rcParams["axes.prop_cycle"].by_key()["color"][:5]
   
    for model in models:
        errors = []
        memory_points = []
        for memory_budget in memory_budgets_folders:
            is_baseline = False
            error = scores[model][memory_budget]["EWEA"]
            errors.append(error)
            mem_point = float(memory_budget.split('_')[1][:-3]) # hard-coded based on folders names
            memory_points.append(mem_point)
                
        if "es" in model:
            f_label = "ElasticSketch"
            marker = "o"
            ls = "-."
            c = orange
        elif "baseline" in model:
            f_label = "CMS"
            marker = "s"
            ls = "--"
            c = blue
        elif "coda" in model:
            f_label = f"(w/ hints) {coda_name}"
            marker = "*"
            ls= "-"
            c = green
        elif "oracle" in model:
            f_label = "(w/ hints) oracle"
            marker = "."
            ls= "-"
        elif "pheavy" in model:
            f_label = "(w/ hints) pHeavy 5pk"
            marker = "."
            ls= ":"
            c = red
            
        ax.plot(memory_points, errors, label=f_label, ls=ls, marker=marker, markersize=10, c=c)

        if legend:
            ax.legend(loc="lower left", frameon=False, borderaxespad=0.05, handletextpad=0.5, ncol=ncol)
        ax.set_xlabel("Memory [MB]")
        ax.set_ylabel("AWAE")
        ax.set_xticks(memory_points)
        ax.set_ylim([0, None])
        
    return ax

In [ ]:
traces_l = []
hh_frac, trace_fmt, minutes = 0.02, 'traces/caida/20160121-13{}00.UTC.anon.pcap', [46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
traces_l.append((hh_frac, trace_fmt, minutes))
hh_frac, trace_fmt, minutes = 0.015, 'traces/mawi/{}.pcap', [1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925]
traces_l.append((hh_frac, trace_fmt, minutes))
hh_frac, trace_fmt, minutes = 0.03, 'traces/uni/{}.pcap', [146, 147, 148, 149, 151, 152, 153, 154, 155, 156]
traces_l.append((hh_frac, trace_fmt, minutes))

proto = 'all'

scores_d = {}
for hh_frac, trace_fmt, minutes in traces_l:
    traces = [trace_fmt.format(mm) for mm in minutes]
    scores = {}
    for trace in traces:
        trace_set, trace_name = trace.split('/')[-2:]
        trace_id = f"{trace_set}-{trace_name.split('.')[0]}"
        folder = f'{results_dir}/{trace_id}/{proto}/top_{hh_frac}_pct/fse/'
        for model in models:
            try:
                scores.setdefault(model, {})
                for i, memory_budget in enumerate(memory_budgets_folders):
                    if trace_set == "uni":
                        actual_memory_budget = memory_budgets_folders_uni[i]
                    else:
                        actual_memory_budget = memory_budget
                    with open(f"{folder}{actual_memory_budget}{'error/'}{model}.txt", 'r') as f:
                        error = float(re.search(r'Total Error:\s+([\d\.e-]+)', f.read()).group(1))
                        scores[model].setdefault(actual_memory_budget, {"EWEA": []})
                        scores[model][actual_memory_budget]["EWEA"].append(error)
            except FileNotFoundError as ex:
                print(ex)
    scores_d[trace_set] = scores

In [ ]:
# compute averages
for trace_set in scores_d:
    for model in scores_d[trace_set]:
        for memory_budget in scores_d[trace_set][model]:
            scores_d[trace_set][model][memory_budget]["EWEA"] = np.mean(scores_d[trace_set][model][memory_budget]["EWEA"])

In [ ]:
fig = plt.figure(figsize=(6, 4.5))
gs = GridSpec(4, 1, height_ratios=[1, 1, 0.17, 0.93], hspace=0.06)

ax = []
ax.append(fig.add_subplot(gs[0]))
ax.append(fig.add_subplot(gs[1], sharex=ax[0]))
ax.append(fig.add_subplot(gs[3]))

ax[0] = plot_errors(
    ax[0], 
    [f for f in models if "oracle" not in f], 
    [m for m in memory_budgets_folders if not "0.5" in m], 
    scores_d['caida'],
    legend=False
)
ax[1] = plot_errors(
    ax[1], 
    [f for f in models if "oracle" not in f], 
    [m for m in memory_budgets_folders if not "0.5" in m], 
    scores_d['mawi'],
    legend=False
    
)
ax[2] = plot_errors(
    ax[2], 
    [f for f in models if "oracle" not in f], 
    [m for m in memory_budgets_folders_uni if not "0.5" in m], 
    scores_d['uni'],
    legend=False
)


box = ax[2].get_position()
ax[2].set_position([box.x0, box.y0, box.width, box.height * 0.3])

ax[0].set_xlabel(None)
ax[1].set_xlabel(None)
ax[0].set_ylabel(None)
ax[1].set_ylabel(None)
ax[2].set_ylabel(None)
ax[0].set_ylim([5, None])
ax[1].set_ylim([0, None])

ax[0].tick_params(direction='in', right=True)
ax[1].tick_params(direction='in', right=True)
ax[2].tick_params(direction='in', right=True)

ax[0].tick_params(axis='x', which='both', bottom=False, labelbottom=False)

fig.text(0.03, 0.57, 'AWAE', va='center', ha='center', rotation='vertical', fontsize=plt.rcParams['axes.labelsize'])

ax[0].text(0.97, 0.9, 'CAIDA', color='black', va='top', ha='right', size=16, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'), transform=ax[0].transAxes)
ax[1].text(0.97, 0.9, 'MAWI', color='black', va='top', ha='right', size=16, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'), transform=ax[1].transAxes)
ax[1].set_yticks([1, 2, 3], [1, 2, 3])
ax[2].text(0.97, 0.9, 'UNI', color='black', va='top', ha='right', size=16, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'), transform=ax[2].transAxes)
ax[2].set_yticks([0, 5, 10], [0, 5, 10])


handles, labels = plt.gca().get_legend_handles_labels()
order = [3, 2, 0, 1,]
ax[0].legend(
    [handles[idx] for idx in order], [labels[idx] for idx in order],
    loc="lower center", 
    frameon=False, 
    borderaxespad=0.05, 
    handletextpad=0.5, 
    ncol=2, 
    bbox_to_anchor=(0.5, 0.95)
)

plt.subplots_adjust(left=0.1, right=0.99, bottom=0.12, top=0.88)
plt.savefig(f'./out/fse.stacked.{proto}.w-uni.pdf', format='pdf', dpi=1200)
plt.show()